# Necessary libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import lagrange
from scipy.interpolate import CubicSpline, InterpolatedUnivariateSpline, Akima1DInterpolator
from datetime import datetime, timedelta
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
import statsmodels.api as sm
#from datetime import timedelta
#import datetime
#import fbprophet
#from matplotlib import pyplot
from pandas import to_datetime
from statsmodels.tsa.arima_model import ARIMA
#from fbprophet import Prophet
#from pmdarima.arima import auto_arima
#import pmdarima
from scipy import interpolate
#from fbprophet import Prophet
from matplotlib import pyplot
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras import Sequential, layers, callbacks
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
import plotly.graph_objs as go
import sklearn.model_selection
from scipy import stats
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow import keras
from imblearn.over_sampling import SMOTE
from scipy.interpolate import CubicSpline as CS

2023-05-11 15:04:11.484517: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-11 15:04:11.484536: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
df=pd.read_csv("3_imputed_data_for_cleaning.csv")
df=df.drop(['Unnamed: 0'],axis=1)
df

,PatientID,DateAdded_New,WoundNumber,Length,Width,Age,BMI,SmokingStatus,Area,PatientGender_new,RaceCodeList_new,patientEthnicity_new
0,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
1,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
2,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
3,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
4,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
...,...,...,...,...,...,...,...,...,...,...,...,...
1726829,10219057,2023-01-09,2,4.05,6.11,73.30,44.600000,3.0,24.7455,M,White,Non-Hispanic
1726830,10219057,2023-01-09,2,4.16,5.59,73.30,44.600000,3.0,23.2544,M,White,Non-Hispanic
1726831,10225856,2023-01-10,1,1.36,0.72,84.90,19.700000,3.0,0.9792,M,White,Non-Hispanic
1726832,10223584,2023-01-11,1,4.18,3.56,42.53,34.500000,1.0,14.8808,F,White,Non-Hispanic


In [3]:
list_unique_patients=df["PatientID"].unique().tolist()
print("Number of unique patients in the dataset",len(list_unique_patients))

Number of unique patients in the dataset 7399


In [ ]:
#defining the dataframe
Wound_assessment=df

#defining the list which has all the unique patients' PatientID
list_patients=Wound_assessment["PatientID"].unique()

#defining to list the patients' index who will be used for forecasting and who will not be used for forecasting
patients_for_forecasting=[]
patients_not_for_forecasting=[]

#defining a variable where we want to count and store the number of total unique wounds for which we want to do the forecasting
unique_wound_number=0

for i in range(len(list_patients)):
    #print(i)
#     if i>9:
#         break
    patient=list_patients[i]
    df_patient=Wound_assessment.query("PatientID == @patient")
    if df_patient.shape[0]>1:
        patients_for_forecasting.append(list_patients[i])
        #print(i)
        wounds_of_list_patient=df_patient["WoundNumber"].unique().tolist()
        #print(wounds_of_list_patient)
        indivi_wound=0
        for j in range(len(wounds_of_list_patient)):
            wound_number= wounds_of_list_patient[j]
            df_patient_1_wound=df_patient.query("WoundNumber==@wound_number")
            #display(df_patient_1_wound)
            if df_patient_1_wound.shape[0]>1:
                if df_patient_1_wound.dropna().shape[0]>=1:
                    indivi_wound=indivi_wound+1
        print("For patient",i+1,", number of unique wounds for which forecasting can be done:",indivi_wound)
#                     #

#                     w_length=pd.Series(df_patient_1_wound["Length"].values)
#                     y = w_length.dropna().values
#                     x=w_length.dropna().index.values
#                     cs = CS(x,y)
#                     x1=w_length.index.values
#                     y1=cs(w_length.index.values)
#                     df_patient_1_wound["Length"]=y1


#                     w_width=pd.Series(df_patient_1_wound["Width"].values)
#                     y = w_width.dropna().values
#                     x=w_width.dropna().index.values
#                     cs = CS(x,y)
#                     x1=w_width.index.values
#                     y1=cs(w_width.index.values)
#                     df_patient_1_wound["Width"]=y1
                    
#                     w_depth=pd.Series(df_patient_1_wound["Depth"].values)
#                     y = w_depth.dropna().values
#                     x=w_depth.dropna().index.values
#                     cs = CS(x,y)
#                     x1=w_depth.index.values
#                     y1=cs(w_depth.index.values)
#                     df_patient_1_wound["Depth"]=y1
                    
                    
#                     display(df_patient_1_wound)
        #print("For patient",i,"number of unique wounds",len(wound_list_patient))
        #for j in range(df_patient_specific_wound=
        unique_wound_number=unique_wound_number+indivi_wound
        #list_count_patient_for_forecasting.append(i)
        #display(df_patient)
    else:
        patients_not_for_forecasting.append(list_patients[i])
        print("For patient",i+1,", Forecasting cannot be done because the patient has only one assessment")
print("Total unique wounds:",unique_wound_number)
print("Patients who have more than 1 assessments:",len(patients_for_forecasting))
print("Patients who have just 1 assessment:",len(patients_not_for_forecasting))

### So we have 370 patients for whom we don't have more than 1 wound assessments, thus we will remove those patients' specific wound information from the original dataframe, or better make two new dataframes

In [5]:
df_non=pd.DataFrame()
for i in range(len(patients_not_for_forecasting)):
    #print(i)
    patient=patients_not_for_forecasting[i]
    df_not_for_forecasting=Wound_assessment.query("PatientID == @patient")
    #display(df_not_for_forecasting)
    df_non=pd.concat([df_non,df_not_for_forecasting],axis=0)
df_non=df_non.reset_index()
df_non=df_non.drop(["index"],axis=1)
df_non

,PatientID,DateAdded_New,WoundNumber,Length,Width,Age,BMI,SmokingStatus,Area,PatientGender_new,RaceCodeList_new,patientEthnicity_new
0,6223300,2017-03-07,1,0.60,0.40,61.80,24.300000,0.0,0.2400,M,White,Non-Hispanic
1,6193541,2017-03-16,1,1.00,0.80,86.70,20.800000,4.0,0.8000,F,White,Non-Hispanic
2,6227686,2017-03-21,1,0.70,0.70,80.72,21.000000,4.0,0.4900,F,White,Non-Hispanic
3,6246709,2017-03-21,1,9.00,1.50,5.67,34.264272,4.0,13.5000,M,White,Non-Hispanic
4,6364023,2017-05-10,1,7.00,6.00,82.62,26.000000,3.0,42.0000,M,White,Non-Hispanic
...,...,...,...,...,...,...,...,...,...,...,...,...
365,10223596,2023-01-04,1,0.93,0.61,79.02,26.600000,1.0,0.5673,F,White,Non-Hispanic
366,10218464,2023-01-05,1,8.08,4.29,92.10,17.200000,4.0,34.6632,F,White,Non-Hispanic
367,10225856,2023-01-10,1,1.36,0.72,84.90,19.700000,3.0,0.9792,M,White,Non-Hispanic
368,10223584,2023-01-11,1,4.18,3.56,42.53,34.500000,1.0,14.8808,F,White,Non-Hispanic


### Check for value_counts of patients for whom we cannot do the forecasting

In [6]:
print(df_non["PatientGender_new"].value_counts(),"\n")
print(df_non["RaceCodeList_new"].value_counts(),"\n")
print(df_non["patientEthnicity_new"].value_counts())

M    200
F    170
Name: PatientGender_new, dtype: int64 

White    338
Black     24
ASKU       5
Asian      2
Other      1
Name: RaceCodeList_new, dtype: int64 

Non-Hispanic    361
Hispanic          9
Name: patientEthnicity_new, dtype: int64


### Then we have 7019 patients for whom we have more than 1 wound assessments, thus we will create a separate dataframe using those patients' information from the original dataframe, and use that for further analysis

In [ ]:
#df_for=pd.DataFrame()
indexes_to_remove=[]
for i in range(len(patients_not_for_forecasting)):
    #print(i)
    patient=patients_not_for_forecasting[i]
    df_not_for_forecasting=Wound_assessment.query("PatientID == @patient")
    indexes_to_remove.append(int(df_not_for_forecasting.index.values))
    #display(df_not_for_forecasting)
#     df_non=pd.concat([df_non,df_not_for_forecasting],axis=0)
# df_non=df_non.reset_index()
# df_non=df_non.drop(["index"],axis=1)
indexes_to_remove

In [8]:
df_for=Wound_assessment
df_for=df_for.drop(indexes_to_remove,axis=0)
df_for=df_for.reset_index()
df_for=df_for.drop(["index"],axis=1)
df_for

,PatientID,DateAdded_New,WoundNumber,Length,Width,Age,BMI,SmokingStatus,Area,PatientGender_new,RaceCodeList_new,patientEthnicity_new
0,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
1,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
2,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
3,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
4,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
...,...,...,...,...,...,...,...,...,...,...,...,...
1726459,10223609,2023-01-09,1,1.74,3.35,56.66,34.264272,4.0,5.8290,F,White,Hispanic
1726460,10223609,2023-01-09,1,1.81,3.39,56.66,34.264272,4.0,6.1359,F,White,Hispanic
1726461,10219057,2023-01-09,1,1.44,0.84,73.30,44.600000,3.0,1.2096,M,White,Non-Hispanic
1726462,10219057,2023-01-09,2,4.05,6.11,73.30,44.600000,3.0,24.7455,M,White,Non-Hispanic


In [9]:
df_for.to_csv("df_for.csv")

In [10]:
df_for=pd.read_csv("df_for.csv")
df_for=df_for.drop(["Unnamed: 0"],axis=1)
df_for

,PatientID,DateAdded_New,WoundNumber,Length,Width,Age,BMI,SmokingStatus,Area,PatientGender_new,RaceCodeList_new,patientEthnicity_new
0,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
1,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
2,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
3,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
4,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
...,...,...,...,...,...,...,...,...,...,...,...,...
1726459,10223609,2023-01-09,1,1.74,3.35,56.66,34.264272,4.0,5.8290,F,White,Hispanic
1726460,10223609,2023-01-09,1,1.81,3.39,56.66,34.264272,4.0,6.1359,F,White,Hispanic
1726461,10219057,2023-01-09,1,1.44,0.84,73.30,44.600000,3.0,1.2096,M,White,Non-Hispanic
1726462,10219057,2023-01-09,2,4.05,6.11,73.30,44.600000,3.0,24.7455,M,White,Non-Hispanic


In [11]:
df_for_squeezed=df_for.drop_duplicates(subset='PatientID', keep="first")
print(df_for_squeezed["PatientGender_new"].value_counts(),"\n")
print(df_for_squeezed["RaceCodeList_new"].value_counts(),"\n")
print(df_for_squeezed["patientEthnicity_new"].value_counts())

M    3641
F    3388
Name: PatientGender_new, dtype: int64 

White                                        6292
Black                                         684
Asian                                          24
ASKU                                           19
American Indian or Alaska Native                6
Native Hawaiian or Other Pacific Islander       3
Other                                           1
Name: RaceCodeList_new, dtype: int64 

Non-Hispanic        6937
Hispanic              83
Patient Declined       8
ASKU                   1
Name: patientEthnicity_new, dtype: int64


### Now that we have a dataframe of 7019 patients who have more than one assessments, we want to remove those patients also who have all NULL information in the assessments, like all assessments are NULL

### Now checking for individual wounds per patients and checking if there is any wound for which we have only one assessment. If yes, then we are removing it.

In [ ]:
list_patient=df_for["PatientID"].unique().tolist()

list_index_wound_specific_1_assessment=[]
for i in range(len(list_patient)):
    print(i)
    patient=list_patient[i]
    df_indivi_patient_all_wounds=df_for.query("PatientID == @patient")
    list_wounds=df_indivi_patient_all_wounds["WoundNumber"].unique().tolist()
    for j in range(len(list_wounds)):
        woundnumber=list_wounds[j]
        df_for_forecasting=df_indivi_patient_all_wounds.query("WoundNumber == @woundnumber")
        #display(df_for_forecasting)
        if df_for_forecasting.shape[0]==1:
            list_index_wound_specific_1_assessment.append(int(df_for_forecasting.index.values))
        

In [13]:
print("For",len(list_index_wound_specific_1_assessment),"number of unique wounds we don't have more than 1 assessments, thus we will remove these information from the original dataframe.")

For 798 number of unique wounds we don't have more than 1 assessments, thus we will remove these information from the original dataframe.


In [14]:
df_for=df_for.drop(list_index_wound_specific_1_assessment,axis=0)
df_for=df_for.reset_index()
df_for=df_for.drop(["index"],axis=1)
df_for

,PatientID,DateAdded_New,WoundNumber,Length,Width,Age,BMI,SmokingStatus,Area,PatientGender_new,RaceCodeList_new,patientEthnicity_new
0,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
1,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
2,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
3,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
4,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
...,...,...,...,...,...,...,...,...,...,...,...,...
1725661,10210244,2023-01-05,1,0.94,0.84,87.58,34.264272,3.0,0.7896,M,White,Non-Hispanic
1725662,10223609,2023-01-09,1,1.74,3.35,56.66,34.264272,4.0,5.8290,F,White,Hispanic
1725663,10223609,2023-01-09,1,1.81,3.39,56.66,34.264272,4.0,6.1359,F,White,Hispanic
1725664,10219057,2023-01-09,2,4.05,6.11,73.30,44.600000,3.0,24.7455,M,White,Non-Hispanic


### Now removing those information if for specific patient, specific wound, all assessments are NULL

In [ ]:
list_patient=df_for["PatientID"].unique().tolist()

list_index_wound_specific_all_null_assessment=[]
for i in range(len(list_patient)):
    print(i)
    patient=list_patient[i]
    df_indivi_patient_all_wounds=df_for.query("PatientID == @patient")
    list_wounds=df_indivi_patient_all_wounds["WoundNumber"].unique().tolist()
    for j in range(len(list_wounds)):
        woundnumber=list_wounds[j]
        df_for_forecasting=df_indivi_patient_all_wounds.query("WoundNumber == @woundnumber")
        #df_for_forecasting_1=df_for_forecasting.drop(["Depth"],axis=1)
        df_for_forecasting_1=df_for_forecasting.dropna()
        if df_for_forecasting_1.shape[0]<1:
            display(df_for_forecasting_1)
            for k in range(len(df_for_forecasting.index.values)):
                list_index_wound_specific_all_null_assessment.append(df_for_forecasting.index.values[k])
        

In [16]:
df_for=df_for.drop(list_index_wound_specific_all_null_assessment,axis=0)
df_for=df_for.reset_index()
df_for=df_for.drop(["index"],axis=1)
df_for

,PatientID,DateAdded_New,WoundNumber,Length,Width,Age,BMI,SmokingStatus,Area,PatientGender_new,RaceCodeList_new,patientEthnicity_new
0,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
1,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
2,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
3,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
4,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
...,...,...,...,...,...,...,...,...,...,...,...,...
1725661,10210244,2023-01-05,1,0.94,0.84,87.58,34.264272,3.0,0.7896,M,White,Non-Hispanic
1725662,10223609,2023-01-09,1,1.74,3.35,56.66,34.264272,4.0,5.8290,F,White,Hispanic
1725663,10223609,2023-01-09,1,1.81,3.39,56.66,34.264272,4.0,6.1359,F,White,Hispanic
1725664,10219057,2023-01-09,2,4.05,6.11,73.30,44.600000,3.0,24.7455,M,White,Non-Hispanic


In [17]:
df_for.to_csv("4_df_cleaned_for_outlier_removal.csv")

In [18]:
df_for["PatientID"].nunique()

6856

In [ ]:
wounds=0
list_patients=df_for["PatientID"].unique().tolist()
for i in range(len(list_patients)):
    print(i)
    patient=list_patients[i]
    data_1=df_for.query("PatientID==@patient")
    number=data_1["WoundNumber"].nunique()
    wounds=wounds+number
print("Number of total unique wounds",wounds)

In [ ]:
print(df["PatientGender_new"].value_counts(),"\n")
print(df["RaceCodeList_new"].value_counts(),"\n")
print(df["patientEthnicity_new"].value_counts())